# Rare decay search

In [ ]:
%pylab inline

In [ ]:
import pandas
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.utils.validation import column_or_1d
from collections import OrderedDict
from hep_ml import metrics
from utils import check_correlation

In [ ]:
!pip install hep_ml

# Load dataset and split into training / test

`training.csv` is a mixture of simulated signal, real background.
It has the following columns.

`test.csv` has the following columns:



In [ ]:
train_ada = pandas.read_csv('reference/training.csv', sep=',')
test_ada = pandas.read_csv('reference/test.csv', sep=',', index_col='id')

In [ ]:
print ("Training full sample columns:", ", ".join(train_ada.columns), "\nShape:", train_ada.shape)

In [ ]:
print ("Test full sample columns:", ", ".join(test_ada.columns), "\nShape:", test_ada.shape)
test_ada.head()

# Train simple model using part of the training sample

In [ ]:
train, test = train_test_split(train_ada, train_size=0.7, test_size=0.3, random_state=13)

Let's chose features to train a model

In [ ]:
variables = list(set(train_ada.columns) - {'id', 'signal', 'mass', 'production', 'min_ANNmuon'})
print (variables)

In [ ]:
%%time
clf = AdaBoostClassifier(n_estimators=100, learning_rate=0.01, random_state=13,
                             base_estimator=DecisionTreeClassifier(max_depth=6, 
                                                                   min_samples_leaf=30,
                                                                   max_features=6,
                                                                   random_state=13))
clf.fit(train[variables], train['signal'])

# Check model quality on a half of the training sample


In [ ]:
def plot_metrics(y_true, y_pred):
    """
    Plots the ROC curve
    
    Parameters
    ----------
    y_true : array-like
        The ground-truth
    y_pred : array-like
        The predictions
    """
    
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred)

    plt.plot(fpr, tpr, label='ROC AUC=%f' % roc_auc)
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.legend()
    plt.title("ROC Curve")

In [ ]:
y_pred = clf.predict_proba(test[variables])[:, 1]

plot_metrics(test['signal'], y_pred)
test.shape, y_pred.shape

ROC AUC is just a part of the solution, you also have to make sure that

- the classifier output is not correlated with the mass
- classifier performs similarily on MC and real data of the normalization channel


### Mass correlation check

In [ ]:
df_corr_check = pandas.read_csv("reference/check_correlation.csv")

In [ ]:
df_corr_check.shape

In [ ]:
y_pred = clf.predict(df_corr_check[variables])

In [ ]:
# NOTE: Never used in original code
# NOTE: Uses functions from utils which is not loaded, but declared in the notebook longer down

def efficiencies(features, 
                 thresholds=None, 
                 mask=None, 
                 bins=30, 
                 labels_dict=None,
                 ignored_sideband=0.0,
                 errors=False,
                 grid_columns=2):
        """
        Efficiencies for spectators
        
        Parameters
        ----------
        features : list or None
            A list of strings of the features to use
            If None, then the classifier's spectators is used
        thresholds : list
            List of floats of what thresholds to use
        mask : None or numbers.Number or array-like or str or function(pandas.DataFrame)
            Mask over which data will be used
        bins : int or array-like
            Bins to use in the histogram
        labels_dict : None or OrderedDict
            Name for class label
            If OrderedDict, the format {int: str} is used
            If None then {0: 'bck', '1': 'signal'}
        ignored_sideband : float
            A float between 0 - 1, where the number indicates the percent of plotting data
        errors : bool
            If True then use errorbar, else interpolate function
        grid_columns : int
            Count of columns in the grid
        
        Returns
        -------
        plotting.GridPlot
        """
        mask, data, class_labels, weight = self._apply_mask(mask,
                                                            self._get_features(features), 
                                                            self.target, self.weight)
        labels_dict = self._check_labels(labels_dict, class_labels)
 
        plots = []
        for feature in data.columns:
            for name, prediction in self.prediction.items():
                prediction = prediction[mask]
                eff = OrderedDict()
                for label, label_name in labels_dict.items():
                    label_mask = class_labels == label
                    eff[label_name] = utils.get_efficiencies(prediction[label_mask, label],
                                                             data[feature][label_mask].values,
                                                             bins_number=bins,
                                                             sample_weight=weight[label_mask],
                                                             thresholds=thresholds,
                                                             errors=errors,
                                                             ignored_sideband=ignored_sideband)
 
                for label_name, eff_data in eff.items():
                    if errors:
                        plot_fig = plotting.ErrorPlot(eff_data)
                    else:
                        plot_fig = plotting.FunctionsPlot(eff_data)
                    plot_fig.xlabel = feature
                    plot_fig.ylabel = 'Efficiency for {}'.format(name)
                    plot_fig.title = '{} flatness'.format(label_name)
                    plot_fig.ylim = (0, 1)
                    plots.append(plot_fig)
 
        return plotting.GridPlot(grid_columns, *plots)

In [ ]:
def check_arrays(*arrays):
    """
    Left for consistency, a version of `sklearn.validation.check_arrays`
    
    Parameters
    ----------
    arrays : argument-tuple
        Input object to check / convert
        Arrays with the same length of first dimension
    
    Returns
    -------
    checked_arrays : object
        The converted and validated array
    """
    
    assert len(arrays) > 0, 'The number of array must be greater than zero'
    checked_arrays = []
    shapes = []
    for arr in arrays:
        if arr is not None:
            checked_arrays.append(numpy.array(arr))
            shapes.append(checked_arrays[-1].shape[0])
        else:
            checked_arrays.append(None)
    assert numpy.sum(numpy.array(shapes) == shapes[0]) == len(shapes), 'Different shapes of the arrays {}'.format(
        shapes)
    return checked_arrays

In [ ]:
def get_efficiencies(prediction, 
                     spectator,
                     sample_weight=None,
                     bins_number=20,
                     thresholds=None,
                     errors=False,
                     ignored_sideband=0.0):
    """
    Construct efficiency function dependent on spectator for each threshold
    
    Different score functions available: Efficiency, Precision, Recall, F1Score,
    and other things from sklearn.metrics
    
    Parameters
    ----------
    prediction : list
        List of probabilities
    spectator : list
        List of spectator's values
    sample_weight : None or array-like
        The weight given the of samples
    bins_number : int
        Count of bins for plot
    thresholds : list
        List of prediction's threshold
        (default=prediction's cuts for which efficiency will be [0.2, 0.4, 0.5, 0.6, 0.8])
    errors : bool
        Whether or not to include errors
    ignored_sidebands : float
            A float between 0 - 1, where the number indicates the percent of plotting data 

    Returns
    -------
    result : OrderedDict
        OrderedDict where the keys are the threshold and the values are tuples of arrays of the same length
        If errors is False, the values are on the form (x_values, y_values)
        If errors is True, the values ar on the form (x_values, y_values, y_err, x_err)
    """
    
    prediction, spectator, sample_weight = \
        check_arrays(prediction, spectator, sample_weight)

    spectator_min, spectator_max = weighted_quantile(spectator, [ignored_sideband, (1. - ignored_sideband)])
    mask = (spectator >= spectator_min) & (spectator <= spectator_max)
    spectator = spectator[mask]
    prediction = prediction[mask]
    bins_number = min(bins_number, len(prediction))
    sample_weight = sample_weight if sample_weight is None else numpy.array(sample_weight)[mask]

    if thresholds is None:
        thresholds = [weighted_quantile(prediction, quantiles=1 - eff, sample_weight=sample_weight)
                      for eff in [0.2, 0.4, 0.5, 0.6, 0.8]]

    binner = Binner(spectator, bins_number=bins_number)
    if sample_weight is None:
        sample_weight = numpy.ones(len(prediction))
    bins_data = binner.split_into_bins(spectator, prediction, sample_weight)

    bin_edges = numpy.array([spectator_min] + list(binner.limits) + [spectator_max])
    xerr = numpy.diff(bin_edges) / 2.
    result = OrderedDict()
    for threshold in thresholds:
        x_values = []
        y_values = []
        N_in_bin = []
        for num, (masses, probabilities, weights) in enumerate(bins_data):
            y_values.append(numpy.average(probabilities > threshold, weights=weights))
            N_in_bin.append(numpy.sum(weights))
            if errors:
                x_values.append((bin_edges[num + 1] + bin_edges[num]) / 2.)
            else:
                x_values.append(numpy.mean(masses))

        x_values, y_values, N_in_bin = check_arrays(x_values, y_values, N_in_bin)
        if errors:
            result[threshold] = (x_values, y_values, numpy.sqrt(y_values * (1 - y_values) / N_in_bin), xerr)
        else:
            result[threshold] = (x_values, y_values)
            
    return result

In [ ]:
def weighted_quantile(array, 
                      quantiles,
                      sample_weight=None,
                      array_sorted=False,
                      old_style=False):
    """
    Computing quantiles of an array. 
    
    Unlike the numpy.percentile, this function supports weights,
    but it is inefficient and performs complete sorting.
    
    Parameters
    ----------
    array : array, shape (n_samples,)
        The input distribution
    quantiles : array-like, shape (n_quantiles,)
        Array of floats from range [0, 1] with quantiles of shape 
    sample_weight : None or array-like, shape (n_samples,)
        Optional weights of the samples
    array_sorted : bool
        If True, the sorting step will be skipped
    old_style : bool
        If True, will correct output to be consistent with numpy.percentile.

    Returns
    -------
    np.array, shape (n_quantiles,)
        The values of the percentiles

    Example
    -------
    >>> weighted_quantile([1, 2, 3, 4, 5], [0.5])
    array([ 3.])
    >>> weighted_quantile([1, 2, 3, 4, 5], [0.5], sample_weight=[3, 1, 1, 1, 1])
    array([ 2.])
    """
    
    array = numpy.array(array)
    quantiles = numpy.array(quantiles)
    sample_weight = check_sample_weight(array, sample_weight)
    assert numpy.all(quantiles >= 0) and numpy.all(quantiles <= 1), 'Percentiles should be in [0, 1]'

    if not array_sorted:
        array, sample_weight = reorder_by_first(array, sample_weight)

    weighted_quantiles = numpy.cumsum(sample_weight) - 0.5 * sample_weight
    if old_style:
        # To be convenient with numpy.percentile
        weighted_quantiles -= weighted_quantiles[0]
        weighted_quantiles /= weighted_quantiles[-1]
    else:
        weighted_quantiles /= numpy.sum(sample_weight)
    return numpy.interp(quantiles, weighted_quantiles, array)


In [ ]:
def check_sample_weight(y_true, sample_weight):
    """
    Asserts that the weights and predictions have the same length
    
    Parameters
    ----------
    y_true : array-like, shape (n_samples,)
        The ground-truth
    sample_weight : None or array-like, shape (n_samples,)
        The assigned weights
    
    Returns
    -------
    array-like, shape (n_samples,)
        The input sample_weight if input sample_weight is not None
        An array of ones else
    """
    
    if sample_weight is None:
        return numpy.ones(len(y_true), dtype=numpy.float)
    else:
        sample_weight = numpy.array(sample_weight, dtype=numpy.float)
        assert len(y_true) == len(sample_weight), \
            "The length of weights is different: not {0}, but {1}".format(len(y_true), len(sample_weight))
        return sample_weight

In [ ]:
def reorder_by_first(*arrays):
    """
    Applies the same permutation to all passed arrays.
    
    The order of the permutation is passed as the first array
    
    Parameters
    ----------
    arrays : argument-tuple
        The first element in arrays must be the order
        The arrays must have the same length of first dimension
 
    Returns
    -------
    list
        A list of the input arrays, ordered by the first input array
    """
    
    arrays = check_arrays(*arrays)
    order = numpy.argsort(arrays[0])
    return [arr[order] for arr in arrays]

In [ ]:
class Binner(object):
    """
    Class that helps to split the values into several bins.
    
    Initially an array of values is given, which is then splitted into 'bins_number' equal parts,
    and thus we are computing limits (boundaries of bins).
    """
        
    def __init__(self, values, bins_number):
        """
        Class constructor
        
        Parameters
        ----------
        values : array-like
            The input distribution
        bins_number : int
            Count of bins for plot
        """
        
        percentiles = [i * 100.0 / bins_number for i in range(1, bins_number)]
        self.limits = numpy.percentile(values, percentiles)

    def get_bins(self, values):
        """
        Given the values of feature, compute the index of bin
        
        Parameters
        ----------
        values : array-like, shape (n_samples,)
            The values to get the bin number from
            
        Returns
        -------
        np.array
            The bin numbers
        """
        
        return numpy.searchsorted(self.limits, values)

    def set_limits(self, limits):
        """Change the thresholds inside bins"""
        self.limits = limits

    # NOTE: Explaination of property decorator
    #       https://stackoverflow.com/questions/17330160/how-does-the-property-decorator-work
    @property
    def bins_number(self):
        """
        Returns the number of bins
        
        Returns
        -------
        int
            The total number of bins
        """
        return len(self.limits) + 1

    def split_into_bins(self, *arrays):
        """
        Split data into bins
        
        Parameters
        ----------
        array : argument-tuple
            Data to be splitted
        
        Returns
        -------
        results : list, shape (n_bins,)
            Values corresponding to each bin.
        """
        
        values = arrays[0]
        for array in arrays:
            assert len(array) == len(values), "passed arrays have different length"
        bins = self.get_bins(values)
        result = []
        for b in range(len(self.limits) + 1):
            indices = bins == b
            result.append([numpy.array(array)[indices] for array in arrays])
        return result

In [ ]:
eff = get_efficiencies(y_pred, df_corr_check.mass, thresholds=[0.5]) #, thresholds=[0.2, 0.4, 0.5, 0.6, 0.8])

In [ ]:
eff.keys()

In [ ]:
for label_name, eff_data in eff.items():
    pyplot.plot(eff_data[0], eff_data[1], label="global eff  %.1f" % label_name)
    
pyplot.xlabel('mass')
pyplot.ylabel('Efficiency')
pyplot.legend();

In [ ]:
corr_metric = check_correlation(y_pred, df_corr_check['mass'])
print (corr_metric)

## MC vs Real difference

In [ ]:
df_agreement = pandas.read_csv('reference/check_agreement.csv')

In [ ]:
# NOTE: Never used in original code

def get_ks_metric(df_agree, df_test):
    """
    Returns the Kolmogorov-Smirnov (ks) metric
    
    Parameter
    ---------
    df_agree : DataFrame
        A dataframe containing the agreement data
    df_test : DataFrame
        A dataframe containing the test data
    
    Returns
    -------
    Series
        The series containing the ks distance
    """
    
    sig_ind = df_agree[df_agree['signal'] == 1].index
    bck_ind = df_agree[df_agree['signal'] == 0].index

    mc_prob = numpy.array(df_test.loc[sig_ind]['prediction'])
    mc_weight = numpy.array(df_agree.loc[sig_ind]['weight'])
    data_prob = numpy.array(df_test.loc[bck_ind]['prediction'])
    data_weight = numpy.array(df_agree.loc[bck_ind]['weight'])
    val, agreement_metric = check_agreement_ks_sample_weighted(data_prob, mc_prob, data_weight, mc_weight)
    
    return agreement_metric['ks']

In [ ]:
# NOTE: Never used in original code

def check_agreement_ks_sample_weighted(data_prediction,
                                       mc_prediction,
                                       weights_data,
                                       weights_mc):
    """
    Checks the agreement between the data prediction and monte carlo prediction
    
    Parameters
    ----------
    data_prediction : array-like
        Predictions from the data
    mc_prediction : array-like
        Predictions from the Monte Carlo simulations
    weights_data : array-like
        Weights for the real data
    weights_mc : array-like
        Wight for the Monte Carlo simulation
    
    Returns
    -------
    bool
        Whether or not the ks distance part is less than 0.03
    result : Dict
        Dictionary on the form {'ks': ks_distance, 'ks_part': ks_distance_part}
    """
    
    data_prediction, weights_data = map(column_or_1d, [data_prediction, weights_data])
    mc_prediction, weights_mc = map(column_or_1d, [mc_prediction, weights_mc])

    assert numpy.all(data_prediction >= 0.) and numpy.all(data_prediction <= 1.), 'error in prediction'
    assert numpy.all(mc_prediction >= 0.) and numpy.all(mc_prediction <= 1.), 'error in prediction'

    weights_data = weights_data / numpy.sum(weights_data)
    weights_mc = weights_mc / numpy.sum(weights_mc)

    data_neg = data_prediction[weights_data < 0]
    weights_neg = -weights_data[weights_data < 0]
    mc_prediction = numpy.concatenate((mc_prediction, data_neg))
    weights_mc = numpy.concatenate((weights_mc, weights_neg))
    data_prediction = data_prediction[weights_data >= 0]
    weights_data = weights_data[weights_data >= 0]

    assert numpy.all(weights_data >= 0) and numpy.all(weights_mc >= 0)
    assert numpy.allclose(weights_data.sum(), weights_mc.sum())

    weights_data /= numpy.sum(weights_data)
    weights_mc /= numpy.sum(weights_mc)

    fpr, tpr, _ = roc_curve_splitted(data_prediction, mc_prediction, weights_data, weights_mc)

    Dnm = numpy.max(numpy.abs(fpr - tpr))
    Dnm_part = numpy.max(numpy.abs(fpr - tpr)[fpr + tpr < 1])

    result = {'ks': Dnm, 'ks_part': Dnm_part}
    return Dnm_part < 0.03, result

In [ ]:
df_agreement.columns

In [ ]:
df_agreement[variables].head()

In [ ]:
def compute_ks(data_prediction, mc_prediction, weights_data, weights_mc):
    """
    Compute Kolmogorov-Smirnov (ks) distance between real data predictions cdf and Monte Carlo one.
    
    Parameters
    ----------
    data_prediction : array-like
        The real data predictions
    mc_prediction : array-like
        The Monte Carlo data predictions
    weights_data : array-like
        The real data weights
    weights_mc : array-like
        The Monte Carlo weights
    
    Returns
    -------
    Dnm : float
        The ks distance
    """
    
    assert len(data_prediction) == len(weights_data), 'Data length and weight one must be the same'
    assert len(mc_prediction) == len(weights_mc), 'Data length and weight one must be the same'

    data_prediction, mc_prediction = numpy.array(data_prediction), numpy.array(mc_prediction)
    weights_data, weights_mc = numpy.array(weights_data), numpy.array(weights_mc)

    assert numpy.all(data_prediction >= 0.) and numpy.all(data_prediction <= 1.), 'Data predictions are out of range [0, 1]'
    assert numpy.all(mc_prediction >= 0.) and numpy.all(mc_prediction <= 1.), 'MC predictions are out of range [0, 1]'

    weights_data /= numpy.sum(weights_data)
    weights_mc /= numpy.sum(weights_mc)

    fpr, tpr = __roc_curve_splitted(data_prediction, mc_prediction, weights_data, weights_mc)

    Dnm = numpy.max(numpy.abs(fpr - tpr))
    return Dnm

In [ ]:
def __roc_curve_splitted(data_zero, data_one, sample_weights_zero, sample_weights_one):
    """
    Compute the roc curve with sample weights
    
    Parameters
    ----------
    data_zero : array-like
        Data labeled with 0
    data_one : array-like
        Data labeled with 1
    sample_weights_zero : array-like 
        Weights for 0-labeled data
    sample_weights_one : array-like
        Weights for 1-labeled data

    Returns
    -------
    fpr : np.array
        The false positive rate
    tpr : np.array
        The true positive rate
    """
    
    labels = [0] * len(data_zero) + [1] * len(data_one)
    weights = numpy.concatenate([sample_weights_zero, sample_weights_one])
    data_all = numpy.concatenate([data_zero, data_one])
    fpr, tpr, _ = roc_curve(labels, data_all, sample_weight=weights)
    
    return fpr, tpr

In [ ]:
agreement_probs = clf.predict_proba(df_agreement[variables])[:, 1]

ks = compute_ks(agreement_probs[df_agreement['signal'].values == 0],
                agreement_probs[df_agreement['signal'].values == 1],
                df_agreement[df_agreement['signal'] == 0]['weight'].values,
                df_agreement[df_agreement['signal'] == 1]['weight'].values)

print ('KS metric:', ks, "is OK:", ks < 0.09)

In [ ]:
def plot_ks(X_agreement, y_pred):
    """
    Plot the prediction distribution
    
    Parameters
    ----------
    X_agreement : DataFrame
        DataFrame with the agreement data
        Must include the column "signal"
    y_pred : array-like
        The prediction
    """
    
    sig_ind = X_agreement[X_agreement['signal'] == 1].index
    bck_ind = X_agreement[X_agreement['signal'] == 0].index

    mc_prob = y_pred[sig_ind]
    mc_weight = numpy.array(X_agreement.loc[sig_ind]['weight'])
    
    data_prob = y_pred[bck_ind]
    data_weight = numpy.array(X_agreement.loc[bck_ind]['weight'])
    
    inds = data_weight < 0
    
    mc_weight = numpy.array(list(mc_weight) + list(-data_weight[inds]))
    mc_prob = numpy.array(list(mc_prob) + list(data_prob[inds]))
    
    data_prob = data_prob[data_weight >= 0]
    data_weight = data_weight[data_weight >= 0]
    
    hist(data_prob, weights=data_weight, color='r', histtype='step', density=True, bins=60, label='data')
    hist(mc_prob, weights=mc_weight, color='b', histtype='step', density=True, bins=60, label='mc')
    
    xlabel("prediction")
    legend(loc=2)
    
    show()

In [ ]:
plot_ks(df_agreement, agreement_probs)

### Let's see if adding some noise can improve the agreement

In [ ]:
def add_noise(array, level=0.40, random_seed=34):
    """
    Adds ramdom noise to the input array
    
    Parameters
    ----------
    array : array-like
        The array to add noise to
    level : float
        The signal portion of the signal/noise ratio
    random_seed : int
        The random seed to use
    """
    
    numpy.random.seed(random_seed)
    
    return level * numpy.random.random(size=array.size) + (1 - level) * array

In [ ]:
agreement_probs_noise = add_noise(clf.predict_proba(df_agreement[variables])[:, 1])

In [ ]:
ks_noise = compute_ks(agreement_probs_noise[df_agreement['signal'].values == 0],
                      agreement_probs_noise[df_agreement['signal'].values == 1],
                      df_agreement[df_agreement['signal'] == 0]['weight'].values,
                      df_agreement[df_agreement['signal'] == 1]['weight'].values)

print ('KS metric:', ks_noise, "is OK:", ks_noise < 0.09)

In [ ]:
plot_ks(df_agreement, agreement_probs_noise)

### Check ROC with noise

In [ ]:
test.shape

In [ ]:
y_pred = add_noise(clf.predict_proba(test[variables])[:, 1])

plot_metrics(test['signal'], y_pred)
test.shape, y_pred.shape

# Train the model using the whole training sample

In [ ]:
%time clf.fit(train_ada[variables], train_ada['signal'])

Compute prediction and add noise

In [ ]:
y_pred = add_noise(clf.predict_proba(test_ada[variables])[:, 1])

# Prepare submission file

In [ ]:
def save_submission(y_pred, index, filename='result'):
    """
    Saves the submission to a csv.gz file
    
    Parameters
    ----------
    y_pred : array-like
        The prediction
    index : array-like
        The id-index corresponding to the prediction
    filename : str
        The base name of the submission file (i.e. excluding the extension)
    
    Returns
    -------
    filename : str
        The file name of the submission file
    """
    
    sep = ','
    filename = '{}.csv.gz'.format(filename)
    pandas.DataFrame({'id': index, 
                      'prediction': y_pred}).to_csv(filename, 
                                                    sep=sep, 
                                                    index=False,
                                                    compression='gzip')
    print ("Saved file: ", filename, "\nShape:", (y_pred.shape[0], 2))
    return filename

In [ ]:
save_submission(y_pred, test_ada.index, "sample_submission")